# <u>Sujet 1 : Evaluation de deux plateformes open source d'analyse linguistique</u>

## <u> I - Evaluation de l'analyse morpho-syntaxique </u>

##### 1) Utiliser le corpus annoté « pos_reference.txt.lima » pour extraire les phrases ayant servi pour produire ce corpus annoté et sauvegarder le résultat dans le fichier « pos_test.txt  ».

In [132]:
def getText(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    for line in fileI:

        tuples = line.split()
        if tuples:
            # On souhaite récupérer l'entièreté de la ligne à l'exception du dernier mot
            # qui est l'annotation
            for i in range(len(tuples)-1):
                fileO.write(tuples[i]+" ")
        else:
            # Si on arrive à une ligne vide on saute une ligne c'est un changement de phrase 
            fileO.write("\n")

    fileI.close()
    fileO.close()


In [133]:
fileIn = "pos_reference.txt.lima"
fileOut = "pos_test.txt"

getText(fileIn,fileOut)


##### 2) Convertir  les  tags  du  corpus  annoté  « pos_reference.txt.lima »  en  tags  universels  et sauvegarder le résultat dans le fichier « pos_reference.txt.univ ». Il faut utiliser les deux tables de correspondance « POSTags_LIMA_PTB_Linux.txt » et « POSTags_PTB_Universal_Linux.txt » pour réaliser cette transformation (LIMA => PTB => Universal). 
 

In [134]:
def setTagUniv(fileIn, fileRef1,fileRef2, fileOut):
    fileI = open(fileIn,'r')
    fileR1 = open(fileRef1,'r')
    fileR2 = open(fileRef2,'r')
    fileO = open(fileOut,'w')

    my_dico1 = {}
    my_dico2 = {}

    #on crée deux dictionnaires à partir des tags de reference
    for line in fileR1:
        tuples = line.split()
        my_dico1[tuples[0]] = tuples[1]
    fileR1.close()

    for line in fileR2:
        tuples = line.split()
        my_dico2[tuples[0]] = tuples[1]
    fileR2.close()

    # modification des tags vers les tags univ
    for l in fileI:
        tup = l.split()
        if tup:
            # on écrit le groupe de mots initial
            for i in range(len(tup)-1):
                    fileO.write(tup[i]+" ")
            temp = my_dico1[tup[len(tup)-1]]

            fileO.write('\t'+ my_dico2[temp])
            fileO.write('\n')
        else:
            fileO.write("\n")

    fileI.close()
    fileO.close()

In [135]:
fileIn = "pos_reference.txt.lima"
fileRef1 = "POSTags_LIMA_PTB_Linux.txt"
fileRef2 = "POSTags_PTB_Universal_Linux.txt"
fileOut = "pos_reference.txt.univ"


setTagUniv(fileIn,fileRef1,fileRef2,fileOut)


##### 3) Lancer les deux POS taggers sur le fichier « pos_test.txt ». Les résultats doivent avoir le format du  corpus  annoté  « pos_reference.txt.lima »  (2  colonnes  séparées  par  une  tabulation)  et doivent être sauvegardés respectivement dans les fichiers suivants : 
-  pos_test.txt.pos.stanford 
-  pos_test.txt.pos.nltk 

In [136]:
import nltk
from nltk.tokenize import word_tokenize

def nltkTokenize(fileIn,fileOut):
    
    # Ouvrir le fichier en lecture seule
    file = open(fileIn, "r")
    fileO = open(fileOut, "w")
    for line in file:
        
        text = word_tokenize(line)
    
        tokens_tag = nltk.pos_tag(text)
        
        for pair in tokens_tag :
            fileO.write('\t'.join(pair))  
            fileO.write('\n')
    fileO.close()
    file.close()   

Ligne de commande à taper dans le terminal linux depuis le dossier stanford-postagger

./stanford-postagger.sh models/english-left3words-distsim.tagger ../pos_test.txt > ../pos_test.txt.pos.stanford

In [137]:
fileIn = "pos_test.txt"
fileOutNltk = "pos_test.txt.pos.nltk"
fileOutStan = "pos_test.txt.pos.stanford"


nltkTokenize(fileIn,fileOutNltk)


##### 4) Convertir les résultats des deux POS taggers en utilisant les étiquettes universelles (Annexe 1). Il faut utiliser la table de correspondance « POSTags_PTB_Universal_Linux.txt » pour réaliser cette  transformation  (PTB => Universal).  Les  résultats  de  cette  conversion  doivent  être sauvegardés respectivement dans les fichiers suivants : 
-  pos_test.txt.pos.stanford.univ 
-  pos_test.txt.pos.nltk.univ

In [138]:
# Création du dictionnaire PTB vers Universal
my_dico = {}

#on crée un dictionnaire à partir des tags de reference
fileR = open(fileRef2,'r')
for line in fileR:
    tuples = line.split()
    my_dico[tuples[0]] = tuples[1]
fileR.close()




In [139]:
def setTagPTBUnivNltk(fileIn, dico, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    
    # modification des tags vers les tags univ
    for l in fileI:
        tup = l.split()
        if tup:
            # on écrit le groupe de mots initial
            fileO.write(tup[0]+'\t'+ dico[tup[1]])
           
            fileO.write('\n')
        else:
            fileO.write("\n")

    fileI.close()
    fileO.close()

In [140]:
setTagPTBUnivNltk(fileOutNltk, my_dico, "pos_test.txt.pos.nltk.univ")

##### Ajout d'étiquettes dans le fichier PTB vers Univ car nous avons constater un manque pour les étiquettes suivantes :
-  $ => CUR
-  `` => .
-  '' => .
-  -LRB- => .
-  -RRB- => .

#### Modification d'un passage dans pos_reference.txt.lima qui fait planter le pos tagger de stanford

 - ligne 3293 => 3 1/2 NUM    ->  3 NUM

In [141]:
def setTagPTBUnivStan(fileIn, dico, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    checkTag = 0
    w = ""
    for l in fileI:
        tup = l.split()
        for word in tup:
            for letter in word:
                if(letter == "_"):
                    fileO.write("\t")
                    checkTag = 1
                if(checkTag == 1 and letter != "_"):
                    w = w+letter
                if(letter != "_" and checkTag == 0):
                    fileO.write(letter)
            fileO.write(" "+dico[w]+" ")
            fileO.write('\n')
            checkTag = 0
            w = ""
    fileI.close()
    fileO.close()   
    

In [142]:
setTagPTBUnivStan(fileOutStan,my_dico,"pos_test.txt.pos.stanford.univ")

##### 5) Lancer l’évaluation des deux POS taggers. Pour réaliser cette évaluation, il faut supprimer la ligne vide (séparant les phrases) dans le de fichier de référence « pos_reference.txt.univ » : 
-  python evaluate.py pos_test.txt.pos.stanford.univ pos_reference.txt.univ 
-  python evaluate.py pos_test.txt.pos.nltk.univ pos_reference.txt.univ 

In [143]:
def suppressEmptyLine(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    for line in fileI:
        tup = line.split()
        if tup:
            fileO.write(line)
    fileI.close()
    fileO.close() 


##### L'objectif est mainteant de se débarasser de cette erreur

Warning: the reference and the candidate consists of different number of lines!

In [144]:
def adaptData(fileIn, fileOut):
    # compte le nombre de mot sur la ligne et recréer des lignes vides ensuite
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    
    for line in fileI:
        tup = line.split(" ")
        tupTab = line.split("\t")
        spaces = len(tup)
        fileO.write(line)
        if(line.__contains__("'s")):
            fileO.write("remplissage "+tupTab[len(tupTab)-1])
        elif(line.__contains__("Toys ``R\" Us Inc.")):
            fileO.write("remplissage "+tupTab[len(tupTab)-1])
            fileO.write("remplissage "+tupTab[len(tupTab)-1])
        elif(line[len(tup[0])-1] == "'"):
            fileO.write("remplissage "+tupTab[len(tupTab)-1])
        
        for i in range (0, spaces-2):
            fileO.write("remplissage "+tupTab[len(tupTab)-1])
    fileI.close()
    fileO.close()

In [145]:
fileRefInter = "pos_reference.txt.univ.modif"
fileRef = "pos_reference.txt.univ"
fileRefOut = "pos_reference.txt.univ.adjustLines"

suppressEmptyLine(fileRef, fileRefInter)
adaptData(fileRefInter,fileRefOut)

##### Ligne de commande à taper pour réaliser les evaluate
-  python evaluateBis.py pos_test.txt.pos.stanford.univ pos_reference.txt.univ
-  python evaluateBis.py pos_test.txt.pos.nltk.univ pos_reference.txt.univ

##### 6) Quelles conclusions vous pouvez avoir à partir des résultats d’évaluation des deux POS taggers. 

## <u> II. Evaluation de la reconnaissance d’entités nommées </u>

##### 1) Utiliser le corpus annoté « ne_reference.txt.conll » pour extraire les phrases ayant servi pour produire ce corpus annoté et sauvegarder le résultat dans le fichier « ne_test.txt ». Dans ce corpus, une ligne vide indique la fin de la phrase courante. 
 

In [146]:
def getTextPartieDeux(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    for line in fileI:

        tuples = line.split()
        if tuples:
            fileO.write(tuples[0]+" ")
        else:
            # Si on arrive à une ligne vide on saute une ligne c'est un changement de phrase 
            fileO.write("\n")

    fileI.close()
    fileO.close()

In [147]:
fileIn = "ne_reference.txt.conll.txt"
fileOut = "ne_test.txt"
getTextPartieDeux(fileIn,fileOut)

##### 2) Lancer les deux NE recognizers sur le fichier « ne_test.txt ». Les résultats doivent avoir le format  du  corpus  annoté  « ne_reference.txt.conll »  (2  colonnes  séparées  par  une tabulation) et doivent être sauvegardés respectivement dans les fichiers suivants : 
-  ne_test.txt.ne.stanford 
-  ne_test.txt.ne.nltk 

In [148]:
import nltk
from nltk.tokenize import word_tokenize

def nltkTokenizeNE(fileIn,fileOut):
    
    file = open(fileIn, "r")
    fileO = open(fileOut, "w")
    for line in file:
        
        text = word_tokenize(line)
    
        tokens_tag = nltk.pos_tag(text)

        ne_tree = nltk.ne_chunk(tokens_tag, binary= False)
        
        for tree in ne_tree.subtrees() :
            
            if tree.label() == "PERSON":
                for branch in tree:
                    b1 = str(branch[0])
                    fileO.write(b1+" ")
                fileO.write("PERS")
                fileO.write("\n")
            elif tree.label() == "ORGANIZATION" or tree.label() == "FACILITY":
                for branch in tree:
                    b1 = str(branch[0])
                    fileO.write(b1+" ")
                fileO.write("ORG")
                fileO.write("\n")
            elif tree.label() == "LOCATION" or tree.label() == "GPE" or  tree.label() == "GSP" :
                for branch in tree:
                    b1 = str(branch[0])
                    fileO.write(b1+" ")
                fileO.write("LOC")
                fileO.write("\n")
            elif tree.label() == "MONEY" or tree.label() == "PERCENT" or tree.label() == "TIME" or  tree.label() == "DATE":
                for branch in tree:
                    b1 = str(branch[0])
                    fileO.write(b1+" ")
                fileO.write("MISC")
                fileO.write("\n")
    fileO.close()
    file.close()     

In [149]:
fileIn = "ne_test.txt"
fileOut = "ne_test.txt.ne.nltk"

nltkTokenizeNE(fileIn, fileOut)

On tape la commande suivante dans un terminal pour appliquer le NE recognizer de standford sur "ne_test.txt" :

java -mx600m -cp stanford-ner.jar:lib/* edu.stanford.nlp.ie.crf.CRFClassifier -loadClassifier classifiers/english.all.3class.distsim.crf.ser.gz -textFile  ../ne_test.txt > ../ne_test.txt.ne.stanford

In [150]:
# Faire une fonction pour que le le texte soit écrit en 2 colonnes
def miseEnPage(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    for l in fileI:
        tup = l.split()
        for word in tup:
            for letter in word:
                if(letter == "/"):
                    fileO.write("\t")
                else : 
                    fileO.write(letter)
            fileO.write('\n')
    fileI.close()
    fileO.close()

In [151]:
fileIn = "ne_test.txt.ne.stanford"
fileOut = "ne_test.txt.ne.stanford.final"
miseEnPage(fileIn,fileOut)

##### 3) Convertir  les  résultats  des  deux  NE  recognizers  en  utilisant  les  étiquettes  CoNLL-2003 (https://www.clips.uantwerpen.be/conll2003/ner/  -Annexe  2-).  Les  résultats  de  cette conversion doivent être sauvegardés respectivement dans les fichiers suivants : 
-  ne_test.txt.ne.stanford.conll 
-  ne_test.txt.ne.nltk.conll 

In [152]:
### Fonction qui permet de convertir les étiquettes de Stanford NE en CoNLL_2003
# Current tag permet de récupérer l'étqiuette courante du token qui précède 
# Si les deux sont identiques on peut mettre que l'on est dans la partie I (inside) de notre entitée nommée
# Sinon on chnage d'étiquette et donc on est au début de la nouvelle entitée nommée (Begining)
# De la même façon que dans la référence nous ne gérons pas le fait que plusieurs entitées nommées puissent se suivre et avoir la même étiquette
#  Exemple : Maria Bara Victor Hugo vont être étiquetés comme étant B-PERS pour Maria et I-PERS pour le reste
#  ###

def convertCoNllStan(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    currentTag =''
    for l in fileI:
        tup = l.split()
        if(tup[1]!= currentTag):
            if(tup[1] == "PERSON"):
                fileO.write(tup[0]+"\t"+"B-PERS"+"\n")
                currentTag = tup[1]
            elif(tup[1] == "LOCATION"):
                fileO.write(tup[0]+"\t"+"B-LOC"+"\n")
                currentTag = tup[1]
            elif(tup[1] == "ORGANIZATION"):
                fileO.write(tup[0]+"\t"+"B-ORG"+"\n")
                currentTag = tup[1]
            elif(tup[1] == "MISC"):
                fileO.write(tup[0]+"\t"+"B-MISC"+"\n")
                currentTag = tup[1]
            else:
                fileO.write(tup[0]+"\t"+tup[1]+"\n")
                currentTag = tup[1]
        else :
            if(tup[1] == "PERSON"):
                fileO.write(tup[0]+"\t"+"I-PERS"+"\n")
                currentTag = tup[1]
            elif(tup[1] == "LOCATION"):
                fileO.write(tup[0]+"\t"+"I-LOC"+"\n")
                currentTag = tup[1]
            elif(tup[1] == "ORGANIZATION"):
                fileO.write(tup[0]+"\t"+"I-ORG"+"\n")
                currentTag = tup[1]
            elif(tup[1] == "MISC"):
                fileO.write(tup[0]+"\t"+"I-MISC"+"\n")
                currentTag = tup[1]
            else:
                fileO.write(tup[0]+"\t"+tup[1]+"\n")
                currentTag = tup[1]
    fileI.close()
    fileO.close()
    

In [153]:
fileOutStan = "ne_test.txt.ne.stanford.conll"
fileInStan = "ne_test.txt.ne.stanford.final"

convertCoNllStan(fileInStan, fileOutStan)

In [154]:
### Fonction qui permet de convertir les étiquettes de NLTK NE en CoNLL_2003
# Current tag permet de récupérer l'étqiuette courante du token qui précède 
# Si les deux sont identiques on peut mettre que l'on est dans la partie I (inside) de notre entitée nommée
# Sinon on chnage d'étiquette et donc on est au début de la nouvelle entitée nommée (Begining)
# De la même façon que dans la référence nous ne gérons pas le fait que plusieurs entitées nommées puissent se suivre et avoir la même étiquette
#  Exemple : Maria Bara Victor Hugo vont être étiquetés comme étant B-PERS pour Maria et I-PERS pour le reste
#
# La différence principale avec la fonction précédente est que l'on peut avoir des groupes de mots dans le même token qu enous avons traités comme étant B-TAG pour le premier et I-TAG pour la suite
#  ###

def convertCoNllNltk(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    currentTag =''
    for l in fileI:
        tup = l.split()
        spaces=len(tup)-1
        if(tup[spaces]!= currentTag):
            if(tup[spaces] == "PERS"):
                fileO.write(tup[0]+"\t"+"B-PERS"+"\n")
                if(spaces>1):
                   for i in range (1,spaces):
                        fileO.write(tup[i]+"\t"+"I-PERS"+"\n")
                currentTag = tup[spaces]
            elif(tup[spaces] == "LOC"):
                fileO.write(tup[0]+"\t"+"B-LOC"+"\n")
                if(spaces>1):
                       for i in range (1,spaces):
                        fileO.write(tup[i]+"\t"+"I-LOC"+"\n")
                currentTag = tup[spaces]
            elif(tup[spaces] == "ORG"):
                fileO.write(tup[0]+"\t"+"B-ORG"+"\n")
                if(spaces>1):
                       for i in range (1,spaces):
                        fileO.write(tup[i]+"\t"+"I-ORG"+"\n")
                currentTag = tup[spaces]
            elif(tup[spaces] == "MISC"):
                fileO.write(tup[0]+"\t"+"B-MISC"+"\n")
                if(spaces>1):
                       for i in range (1,spaces):
                        fileO.write(tup[i]+"\t"+"I-MISC"+"\n")
                currentTag = tup[spaces]
            else:
                fileO.write(tup[0]+"\t"+tup[spaces]+"\n")
                if(spaces>1):
                       for i in range (1,spaces):
                        fileO.write(tup[i]+"\t"+"O"+"\n")
                currentTag = tup[spaces]
        else :
            if(tup[spaces] == "PERS"):
                fileO.write(tup[0]+"\t"+"I-PERS"+"\n")
                currentTag = tup[spaces]
            elif(tup[spaces] == "LOC"):
                fileO.write(tup[0]+"\t"+"I-LOC"+"\n")
                currentTag = tup[spaces]
            elif(tup[spaces] == "ORG"):
                fileO.write(tup[0]+"\t"+"I-ORG"+"\n")
                currentTag = tup[1]
            elif(tup[spaces] == "MISC"):
                fileO.write(tup[0]+"\t"+"I-MISC"+"\n")
                currentTag = tup[spaces]
            else:
                fileO.write(tup[0]+"\t"+tup[spaces]+"\n")
                currentTag = tup[spaces]
    fileI.close()
    fileO.close()
    

In [155]:
fileInNLTK = "ne_test.txt.ne.nltk"
fileOutNLTK = "ne_test.txt.ne.nltk.conll"

convertCoNllNltk(fileInNLTK, fileOutNLTK)

##### 4) Lancer  l’évaluation  des  deux  NE  recognizers.  Pour  réaliser  cette  évaluation,  il  faut supprimer  la  ligne  vide  (séparant  les  phrases)  dans  le  de  fichier  de  référence « ne_reference.txt.conll »: 
-  python evaluateBis.py ne_test.txt.ne.stanford.conll ne_reference.txt.conll.txt.final
-  python evaluateBis.py ne_test.txt.ne.nltk.conll ne_reference.txt.conll.txt.final

In [129]:
suppressEmptyLine("ne_reference.txt.conll.txt","ne_reference.txt.conll.txt.final")

##### 5) Quelles conclusions vous pouvez avoir à partir des résultats d’évaluation des deux NE recognizers.